In [58]:
import numpy as np
import pandas as pd

import copy

ccol = dict(zip(['pleiades',"hyades","praesepe","ngc6774","ngc2682"],
                [["Pleiades","brown","x",135,25,25],
                 ["Hyades","orangered","*",690,160,100],
                 ["Praesepe","orange","d",750,3,7],
                 ["Rup 147", "green",r"$m$",2650,380,380],
                 ["M67","blue",">",3639,17,17]]))

In [59]:
df = pd.read_csv("../k2scoc/results/tables/full_table.csv")

luminosities = pd.DataFrame()
for cl in ["ngc2682","ngc6774","praesepe","pleiades","hyades"]:
    lum = pd.read_csv(f"../StellarParameters/luminosities/{cl}_luminosities.csv")
    lum = lum[["EPIC","Lum_SED", "e_Lum_SED"]]
    lum.EPIC = lum.EPIC.astype(int)
    lum = lum.drop_duplicates()
    luminosities = luminosities.append(lum)

df = df.merge(luminosities, how="left", on="EPIC")
df.shape

(6254, 26)

In [60]:
df.head(4).T

,0,1,2,3
Unnamed: 0,2109,3662,3791,4514
istart,1874,3248,1546,648
istop,1881,3251,1553,651
cstart,105695,144350,142526,162659
cstop,105703,144353,142534,162663
tstart,2268.66,3058.52,3021.25,3432.64
tstop,2268.83,3058.58,3021.42,3432.73
ed_rec,11162.2,2040.66,207.84,2932.48
ed_rec_err,105.664,82.6106,8.01233,6.87985
ampl_rec,1.27536,0.671565,0.034262,1.00671


In [69]:
df2 = copy.deepcopy(df)
df2["cstart"] = df2.cstart.fillna(0).astype(int)
df2["cstop"] = df2.cstop.fillna(0).astype(int)
#df2["todrop"] = df2.todrop.fillna("")
df2 = df2.rename(index = str, columns={"cstart" : r"$c_0$",
                                     "cstop" : r"$c_1$",
                                     #"todrop" : "phot. flag",
                                     #"real" : "vet. flag",
                                     "ampl_rec" : "$a$",
                                    })
def rounded(ed, ederr):
    if ed>1000:
        ed, ederr = int(np.rint(ed)), int(np.rint(ederr))
    if (ed>100) & (ed<1000):
        ed, ederr = np.round(ed, 1), np.round(ederr, 1),
    if ed < 100:
        ed, ederr = np.round(ed, 2), np.round(ederr, 2) 
    return f"{ed}({ederr})"

df2["cluster"] = df2.cluster.apply(lambda x: ccol[x][0])
df2[r"$ED$ [s]"] = df2.apply(lambda x: rounded(x.ed_rec, x.ed_rec_err), axis=1)
df2["powKep"] = df2.Lum_Kepler.apply(lambda x: int(str(x)[-2:]))
df2["epowKep"] = df2.e_Lum_Kepler.apply(lambda x: int(str(x)[-2:]))
df2["powSED"] = df2.Lum_SED.apply(lambda x: int(str(x)[-2:]))
df2["epowSED"] = df2.e_Lum_SED.apply(lambda x: int(str(x)[-2:]))

df2[r"$L_\mathrm{Kp}$ [erg/s]"] = df2.apply(lambda x: f"${str(x.Lum_Kepler)[:4]}({float(str(x.e_Lum_Kepler)[:4])/np.power(10,x.powKep-x.epowKep):.2f})",axis=1)
df2[r"$L_\mathrm{Kp}$ [erg/s]"] = df2[r"$L_\mathrm{Kp}$ [erg/s]"] + r"10^{" + df2.powKep.astype(str) + r"}$"
df2[r"$L_\mathrm{bol,*}$ [erg/s]"] = df2.apply(lambda x: f"${str(x.Lum_SED)[:4]}({float(str(x.e_Lum_SED)[:4])/np.power(10,x.powSED-x.epowSED):.2f})",axis=1)
df2[r"$L_\mathrm{bol,*}$ [erg/s]"] = df2[r"$L_\mathrm{bol,*}$ [erg/s]"] + r"10^{" + df2.powSED.astype(str) + r"}$"
df2[r"$T_\mathrm{eff}$ [K]"] = df2.apply(lambda x: f"{(int(np.rint(x.Teff_median)))}({(int(np.rint(x.Teff_std)))})", axis=1)
df2.head(5).T        

,0,1,2,3,4
Unnamed: 0,2109,3662,3791,4514,4930
istart,1874,3248,1546,648,1058
istop,1881,3251,1553,651,1061
$c_0$,105695,144350,142526,162659,163090
$c_1$,105703,144353,142534,162663,163093
tstart,2268.66,3058.52,3021.25,3432.64,3441.45
tstop,2268.83,3058.58,3021.42,3432.73,3441.51
ed_rec,11162.2,2040.66,207.84,2932.48,671.607
ed_rec_err,105.664,82.6106,8.01233,6.87985,12.1908
$a$,1.27536,0.671565,0.034262,1.00671,0.261612


In [70]:
df2 = df2[(df2.real==1) & (df2.todrop.isnull())]

In [71]:
df3 = df2[["EPIC", "C", "cluster", r"$c_0$",r"$c_1$", r"$a$", r"$T_\mathrm{eff}$ [K]",
           r"$ED$ [s]", r"$L_\mathrm{bol,*}$ [erg/s]",r"$L_\mathrm{Kp}$ [erg/s]"]]
df3 = df3.sort_values(by=r"$a$", ascending=False).iloc[:30]

nc = 'c' * (df3.shape[1])
stri = df3.to_latex(index=False,escape=False, column_format=f"l{nc}r")
stri = stri.replace("\\toprule","\hline")
stri = stri.replace("\\midrule","\hline")


stri = stri.replace("\\bottomrule","\hline\n" )
stri = stri.replace("{}"," " )

with open(f"/home/ekaterina/Documents/002_writing/flares-in-clusters-ii-draft/Flares_in_Clusters_with_K2_2/tables/flares.tex", "w") as f:
    f.write(stri)
